In [1]:
import os
from collections import namedtuple
import pickle

import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from finta import TA
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.trading.client import TradingClient

from common import trader

database_uri: postgresql://postgres:password@db:5432/smigtraderdb
Request params: {'timeframe': <alpaca.data.timeframe.TimeFrame object at 0x75baeefe7230>, 'start': datetime.datetime(2023, 1, 1, 0, 0), 'end': datetime.datetime(2023, 12, 31, 0, 0), 'symbol_or_symbols': ['AAPL', 'TSLA', 'MSFT', 'FB', 'GOOG', 'AMZN', 'NVDA', 'NFLX', 'SPX', 'QQQ', 'DJI', 'VIX']}


In [4]:
# get data
trader._get_stock_bars()

## Save/Load Data

In [7]:
# save data
with open("data.pkl", "wb") as f:  # 'wb' means write in binary mode
    pickle.dump(trader.data.df, f)

In [10]:
# load data
file = open("../data/data.pkl",'rb')
df = pickle.load(file)
file.close()

In [22]:
from database import get_db
from models import StockObservation

In [19]:
db = next(get_db())

## Load data into Postgres

In [24]:
tickers = df.index.get_level_values('symbol').unique()

for ticker in tickers:
    # Filter the DataFrame by ticker
    ticker_df = df.xs(ticker, level='symbol')
    print(f"Processing data for {ticker}\tshape: {ticker_df.shape}")

    for index, row in ticker_df.iterrows():
        timestamp = index  
        # Create a StockObservation object
        stock_observation = StockObservation(
            symbol=ticker,
            timestamp=timestamp,
            open=row['open'],
            high=row['high'],
            low=row['low'],
            close=row['close'],
            volume=row['volume'],
            trade_count=row['trade_count'],
            vwap=row['vwap'],
        )
        
        # Add the object to the session
        db.add(stock_observation)
    
    # Commit the session to insert into the database
    db.commit()

Processing data for AAPL	shape: (250, 7)


OperationalError: (psycopg2.OperationalError) could not translate host name "db" to address: Name or service not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
bars = stock_data_client.get_stock_bars(request_params)

In [ ]:
df = bars.df
df_reset = df.reset_index()
df_reset["timestamp"] = pd.to_datetime(df_reset["timestamp"])
df = df_reset.set_index("timestamp")

In [ ]:
df.symbol.unique()

In [ ]:
from finta.utils import resample_calendar

weekly_df = resample_calendar(df[df["symbol"] == "AAPL"], "7d")

In [ ]:
TA.EMA(weekly_df, 5).tail(10)

In [ ]:
# calc bol band
bbands = TA.BBANDS(df, 30)

# cherry pick what to show on the chart
bands_plot = pd.concat([bbands.BB_UPPER, bbands.BB_LOWER], axis=1)

apd = mpf.make_addplot(bands_plot.tail(300))

mpf.plot(
    df.tail(300),
    type="candle",
    style="charles",
    title="AAPL BBANDS(30)",
    ylabel="Price (USD)",
    ylabel_lower="Volume",
    volume=True,
    figscale=1.5,
    addplot=apd,
)